# Lab 1: Parallel Function Calling & Error Handling

**Duration:** 90-120 minutes  
**Level:** Intermediate to Advanced

## Learning Objectives

By the end of this lab, you will be able to:
1. Implement parallel function calling with OpenAI API
2. Handle tool execution errors gracefully
3. Build retry mechanisms with exponential backoff
4. Create circuit breakers for failing tools
5. Monitor tool execution with metrics
6. Implement fallback strategies

## Setup

In [ ]:
# Install required packages
!pip install openai tenacity httpx -q

In [ ]:
import os
import json
import time
import random
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✓ Setup complete")

## Exercise 1: Sequential vs Parallel Tool Execution

Compare performance between sequential and parallel function calling.

**Task:** Implement tools that simulate API calls with delays, then measure execution time for sequential vs parallel patterns.

In [ ]:
# Mock tool implementations with delays
def get_weather(location: str) -> dict:
    """Simulate weather API call (2 seconds)"""
    time.sleep(2)
    return {
        "location": location,
        "temperature": random.randint(15, 30),
        "condition": random.choice(["sunny", "cloudy", "rainy"])
    }

def get_stock_price(symbol: str) -> dict:
    """Simulate stock API call (2 seconds)"""
    time.sleep(2)
    return {
        "symbol": symbol,
        "price": round(random.uniform(100, 500), 2),
        "change": round(random.uniform(-5, 5), 2)
    }

def search_news(query: str) -> dict:
    """Simulate news search API call (2 seconds)"""
    time.sleep(2)
    return {
        "query": query,
        "results": [
            {"title": f"News about {query} #{i}", "url": f"http://example.com/{i}"}
            for i in range(3)
        ]
    }

# Map function names to implementations
available_functions = {
    "get_weather": get_weather,
    "get_stock_price": get_stock_price,
    "search_news": search_news,
}

# Define tools for OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get current stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Stock symbol"}
                },
                "required": ["symbol"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_news",
            "description": "Search for news articles",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"}
                },
                "required": ["query"]
            }
        }
    }
]

print("✓ Tools defined")

In [ ]:
# TODO: Implement function to execute tools with OpenAI
def execute_tools_with_timing(user_message: str, parallel: bool = True) -> tuple[str, float]:
    """
    Execute tools and measure execution time.
    
    Args:
        user_message: User's request
        parallel: Enable parallel tool calling
    
    Returns:
        (response_text, execution_time_seconds)
    """
    start_time = time.time()
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to tools."},
        {"role": "user", "content": user_message}
    ]
    
    # TODO: Make initial API call with tools
    # Hint: Use parallel_tool_calls parameter
    response = None  # Replace with your code
    
    message = response.choices[0].message
    
    # TODO: Check if model wants to call functions
    if message.tool_calls:
        messages.append(message)
        
        # TODO: Execute all tool calls
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"  Calling {function_name}({function_args})")
            
            # TODO: Execute function and append result
            pass  # Replace with your code
        
        # TODO: Second API call with tool results
        second_response = None  # Replace with your code
        
        final_message = second_response.choices[0].message.content
    else:
        final_message = message.content
    
    execution_time = time.time() - start_time
    return final_message, execution_time

# Test sequential execution
print("Testing SEQUENTIAL execution...")
response, seq_time = execute_tools_with_timing(
    "What's the weather in London, stock price of AAPL, and latest tech news?",
    parallel=False
)
print(f"Sequential time: {seq_time:.2f}s\n")

# Test parallel execution
print("Testing PARALLEL execution...")
response, par_time = execute_tools_with_timing(
    "What's the weather in London, stock price of AAPL, and latest tech news?",
    parallel=True
)
print(f"Parallel time: {par_time:.2f}s\n")

# Calculate speedup
speedup = seq_time / par_time
print(f"⚡ Speedup: {speedup:.2f}x")

## Exercise 2: Error Handling with Try-Catch

Implement robust error handling for tool execution.

**Task:** Create a tool executor that catches errors and returns structured error information to the LLM.

In [ ]:
# Flaky tool that sometimes fails
def flaky_api_call(location: str) -> dict:
    """API that fails 40% of the time"""
    if random.random() < 0.4:
        raise Exception("API temporarily unavailable")
    return {"location": location, "data": "success"}

# TODO: Implement safe tool execution
def execute_tool_safely(function_name: str, function_args: dict) -> dict:
    """
    Execute tool with error handling.
    
    Returns:
        dict with 'success', 'result' or 'error' fields
    """
    try:
        # TODO: Get function from available_functions
        # TODO: Execute function with arguments
        # TODO: Return success response
        pass
    except TypeError as e:
        # TODO: Handle invalid arguments
        pass
    except Exception as e:
        # TODO: Handle unexpected errors
        pass

# Test error handling
print("Testing error handling...")
available_functions["flaky_api_call"] = flaky_api_call

for i in range(5):
    result = execute_tool_safely("flaky_api_call", {"location": "London"})
    status = "✓" if result.get("success") else "✗"
    print(f"{status} Attempt {i+1}: {result}")

## Exercise 3: Retry Mechanism with Exponential Backoff

Implement automatic retry logic for transient failures.

**Task:** Build a retry decorator that retries failed tool calls with increasing delays.

In [ ]:
# TODO: Implement retry with exponential backoff
def retry_with_backoff(
    func,
    max_attempts: int = 3,
    initial_delay: float = 1.0,
    exponential_base: float = 2.0,
    *args,
    **kwargs
):
    """
    Retry function with exponential backoff.
    
    Args:
        func: Function to retry
        max_attempts: Maximum retry attempts
        initial_delay: Initial delay in seconds
        exponential_base: Multiplier for each retry
    """
    attempt = 0
    delay = initial_delay
    
    while attempt < max_attempts:
        try:
            # TODO: Try to execute function
            pass
        except Exception as e:
            attempt += 1
            
            if attempt >= max_attempts:
                # TODO: Raise exception after max attempts
                pass
            
            # TODO: Calculate backoff delay
            # TODO: Sleep for delay seconds
            print(f"Attempt {attempt} failed: {e}. Retrying in {delay:.2f}s...")

# Test retry mechanism
print("Testing retry mechanism...\n")
try:
    result = retry_with_backoff(flaky_api_call, max_attempts=5, location="Paris")
    print(f"\n✓ Success: {result}")
except Exception as e:
    print(f"\n✗ Failed after all retries: {e}")

## Exercise 4: Using Tenacity Library

Use the `tenacity` library for production-ready retry logic.

**Task:** Apply the `@retry` decorator to handle specific exceptions with configurable backoff.

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    before_sleep_log
)
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# TODO: Decorate function with retry logic
# Hint: Use @retry decorator from tenacity
def call_external_api(endpoint: str) -> dict:
    """Call external API with automatic retry."""
    # Simulate flaky API
    if random.random() < 0.5:
        raise ConnectionError("API connection failed")
    return {"endpoint": endpoint, "status": "success"}

# Test tenacity retry
print("Testing tenacity retry...\n")
try:
    result = call_external_api("/api/data")
    print(f"\n✓ Result: {result}")
except Exception as e:
    print(f"\n✗ Failed: {e}")

## Exercise 5: Circuit Breaker Pattern

Implement a circuit breaker to prevent cascading failures.

**Task:** Build a circuit breaker that opens after repeated failures and automatically recovers.

In [ ]:
class CircuitState(Enum):
    CLOSED = "closed"      # Normal operation
    OPEN = "open"          # Blocking calls
    HALF_OPEN = "half_open"  # Testing recovery

@dataclass
class CircuitBreaker:
    """
    Circuit breaker to prevent repeated failures.
    """
    failure_threshold: int = 5
    timeout_seconds: int = 60
    half_open_max_calls: int = 3
    
    state: CircuitState = field(default=CircuitState.CLOSED, init=False)
    failure_count: int = field(default=0, init=False)
    last_failure_time: Optional[datetime] = field(default=None, init=False)
    half_open_calls: int = field(default=0, init=False)
    
    def call(self, func, *args, **kwargs):
        """
        Execute function through circuit breaker.
        """
        # TODO: Check if circuit is OPEN
        if self.state == CircuitState.OPEN:
            # TODO: Check if timeout has elapsed
            # TODO: If yes, transition to HALF_OPEN
            # TODO: If no, raise exception
            pass
        
        # TODO: Execute function
        try:
            result = None  # Replace with function call
            
            # TODO: On success, reset failure count
            if self.state == CircuitState.HALF_OPEN:
                # TODO: Increment half_open_calls
                # TODO: If reached max, close circuit
                pass
            
            return result
            
        except Exception as e:
            # TODO: On failure, increment failure_count
            # TODO: If threshold reached, open circuit
            raise
    
    def reset(self):
        """Manually reset circuit breaker."""
        self.state = CircuitState.CLOSED
        self.failure_count = 0
        self.half_open_calls = 0
        self.last_failure_time = None

# Test circuit breaker
print("Testing circuit breaker...\n")

def unreliable_service():
    """Service that fails 80% of the time"""
    if random.random() < 0.8:
        raise Exception("Service unavailable")
    return "Success"

breaker = CircuitBreaker(failure_threshold=3, timeout_seconds=5)

# Simulate multiple calls
for i in range(15):
    try:
        result = breaker.call(unreliable_service)
        print(f"✓ Call {i+1}: {result} (state: {breaker.state.value})")
    except Exception as e:
        print(f"✗ Call {i+1}: {e} (state: {breaker.state.value}, failures: {breaker.failure_count})")
    
    time.sleep(0.5)

## Exercise 6: Metrics Collection

Track tool execution metrics for monitoring.

**Task:** Build a metrics collector that tracks success rate, latency, and call counts.

In [ ]:
from collections import defaultdict
from dataclasses import dataclass

@dataclass
class ToolMetrics:
    """Metrics for a tool execution."""
    tool_name: str
    duration_ms: float
    success: bool
    timestamp: datetime

class MetricsCollector:
    """Collect and aggregate tool execution metrics."""
    
    def __init__(self):
        self.metrics: List[ToolMetrics] = []
        self.counters = defaultdict(lambda: {"success": 0, "failure": 0})
        self.latencies = defaultdict(list)
    
    def record(self, tool_name: str, duration_ms: float, success: bool):
        """Record a tool execution."""
        # TODO: Create ToolMetrics and append to list
        # TODO: Update counters
        # TODO: Record latency
        pass
    
    def get_summary(self) -> Dict:
        """Get summary statistics."""
        summary = {}
        
        for tool_name in self.counters:
            # TODO: Calculate success rate
            # TODO: Calculate average latency
            # TODO: Calculate p95 latency
            pass
        
        return summary

# Test metrics collection
print("Testing metrics collection...\n")

metrics = MetricsCollector()

# Simulate tool executions
for i in range(50):
    tool_name = random.choice(["get_weather", "get_stock_price", "search_news"])
    duration = random.uniform(100, 2000)  # ms
    success = random.random() > 0.2  # 80% success rate
    
    metrics.record(tool_name, duration, success)

# Print summary
summary = metrics.get_summary()
print(json.dumps(summary, indent=2))

## Exercise 7: Fallback Strategies

Implement fallback mechanisms when tools fail.

**Task:** Create a system that tries primary tool, falls back to secondary, then cached data.

In [ ]:
# Fallback hierarchy
def get_weather_primary(location: str) -> dict:
    """Primary weather API (flaky)"""
    if random.random() < 0.7:
        raise Exception("Primary API unavailable")
    return {"source": "primary", "location": location, "temp": 22}

def get_weather_secondary(location: str) -> dict:
    """Secondary weather API (less flaky)"""
    if random.random() < 0.3:
        raise Exception("Secondary API unavailable")
    return {"source": "secondary", "location": location, "temp": 21}

def get_weather_cached(location: str) -> dict:
    """Cached weather data (always available)"""
    cache = {
        "London": {"temp": 18, "condition": "cloudy"},
        "Paris": {"temp": 20, "condition": "sunny"},
        "Tokyo": {"temp": 25, "condition": "rainy"},
    }
    return {
        "source": "cache",
        "location": location,
        **cache.get(location, {"temp": None, "condition": "unknown"})
    }

# TODO: Implement cascading fallback
def get_weather_with_fallback(location: str) -> dict:
    """
    Try primary, secondary, then cache.
    
    Returns:
        Weather data with 'source' indicating which succeeded
    """
    # TODO: Try primary API
    # TODO: On failure, try secondary API
    # TODO: On failure, use cached data
    pass

# Test fallback system
print("Testing fallback system...\n")

for i in range(10):
    result = get_weather_with_fallback("London")
    print(f"Attempt {i+1}: {result['source']} - {result}")

## Exercise 8: Complete Production System

Combine all patterns into a production-ready tool executor.

**Task:** Build a tool executor with retry, circuit breaker, metrics, and fallbacks.

In [ ]:
class ProductionToolExecutor:
    """
    Production-ready tool executor with all error handling.
    """
    
    def __init__(self):
        self.circuit_breakers = defaultdict(lambda: CircuitBreaker())
        self.metrics = MetricsCollector()
    
    def execute(
        self,
        tool_name: str,
        function: callable,
        args: dict,
        fallback: Optional[callable] = None
    ) -> dict:
        """
        Execute tool with comprehensive error handling.
        
        Args:
            tool_name: Name of tool
            function: Primary function to execute
            args: Function arguments
            fallback: Optional fallback function
        
        Returns:
            Result with success flag and data/error
        """
        start_time = time.time()
        breaker = self.circuit_breakers[tool_name]
        
        # TODO: Try to execute through circuit breaker
        try:
            # TODO: Call function with retry logic
            result = None  # Replace with your code
            
            # TODO: Record success metrics
            
            return {
                "success": True,
                "result": result,
                "source": "primary"
            }
            
        except Exception as e:
            # TODO: Record failure metrics
            
            # TODO: Try fallback if available
            if fallback:
                try:
                    result = None  # Replace with fallback call
                    return {
                        "success": True,
                        "result": result,
                        "source": "fallback"
                    }
                except Exception as fb_error:
                    pass
            
            # TODO: Return error response
            return {
                "success": False,
                "error": str(e),
                "suggestion": "Try again later or check service status"
            }
    
    def get_stats(self) -> dict:
        """Get execution statistics."""
        return self.metrics.get_summary()

# Test production system
print("Testing production tool executor...\n")

executor = ProductionToolExecutor()

# Simulate 100 tool calls
for i in range(100):
    result = executor.execute(
        tool_name="get_weather",
        function=get_weather_primary,
        args={"location": "London"},
        fallback=lambda location: get_weather_cached(location)
    )
    
    if i % 20 == 0:
        status = "✓" if result["success"] else "✗"
        source = result.get("source", "error")
        print(f"{status} Call {i+1}: {source}")

# Print final stats
print("\n=== Final Statistics ===")
stats = executor.get_stats()
print(json.dumps(stats, indent=2))

## Bonus Exercise: Timeout Handling

Implement timeout mechanism for slow tools.

**Task:** Create a tool wrapper that kills execution after a timeout.

In [ ]:
import signal
from contextlib import contextmanager

@contextmanager
def timeout(seconds: int):
    """
    Context manager for timeout.
    
    Usage:
        with timeout(5):
            slow_function()
    """
    def timeout_handler(signum, frame):
        raise TimeoutError(f"Operation timed out after {seconds}s")
    
    # TODO: Set up signal handler
    # TODO: Set alarm
    # TODO: Yield control
    # TODO: Cancel alarm in finally block
    pass

# Test timeout
def slow_operation():
    """Operation that takes 10 seconds"""
    time.sleep(10)
    return "Completed"

print("Testing timeout mechanism...\n")

try:
    with timeout(3):
        result = slow_operation()
        print(f"✓ Result: {result}")
except TimeoutError as e:
    print(f"✗ Timeout: {e}")

## Summary

### Key Concepts Covered

1. **Parallel vs Sequential**: Parallel execution provides significant speedup for independent tools
2. **Error Handling**: Structured error responses help LLM understand and handle failures
3. **Retry Logic**: Exponential backoff increases success rate for transient failures
4. **Circuit Breakers**: Prevent cascading failures and allow automatic recovery
5. **Metrics**: Track success rates and latency for monitoring
6. **Fallbacks**: Provide degraded service when primary tools fail

### Production Checklist

- [ ] Enable parallel tool calling for independent tools
- [ ] Wrap all tool executions in try-catch
- [ ] Implement retry with exponential backoff
- [ ] Add circuit breakers for external services
- [ ] Collect metrics for all tool calls
- [ ] Define fallback strategies
- [ ] Set timeouts for all operations
- [ ] Return structured errors to LLM
- [ ] Log all failures for debugging
- [ ] Test failure scenarios

### Next Steps

- Lab 2: Building a Tool Registry System
- Lab 3: Multi-Tool Workflow Orchestration
- Lesson 2: Building Production Tool Systems